In [1]:
import numpy as np
import os.path as osp
import scipy.stats as stats

from geo_tool import Point_Cloud

from tf_lab.nips.helper import pclouds_centered_and_half_sphere
from tf_lab.iclr.helper import load_multiple_version_of_pcs
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from pcloud_benchmark.evaluate_gan import entropy_of_occupancy_grid, jensen_shannon_divergence

PyTorch not working. MMD measurement won't be available


In [2]:
%matplotlib inline

In [7]:
# Parameters
class_name = 'chair'
n_pc_samples = 2048
cmp_in_sphere = True
voxel_resolution = 28

In [8]:
# Load GT Data
syn_id = snc_category_to_synth_id()[class_name]
in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)
train_data = in_data['train'].point_clouds
test_data = in_data['test'].point_clouds
val_data = in_data['val'].point_clouds
gt_data = np.vstack([train_data, test_data, val_data])
gt_data = pclouds_centered_and_half_sphere(gt_data)
_, gt_grid_var = entropy_of_occupancy_grid(gt_data, voxel_resolution, in_sphere=cmp_in_sphere)

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt
679 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt
5761 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt
338 pclouds were loaded. They belong in 1 shape-classes.


In [11]:
# Compare JSD of GT with random noise.
lower = -0.5
upper = 0.5
random_pcs = np.random.uniform(low=lower, high=upper, size=(len(gt_data), n_pc_samples, 3))
# random_pcs = pclouds_centered_and_half_sphere(random_pcs)
_, random_grid_var = entropy_of_occupancy_grid(random_pcs, voxel_resolution, in_sphere=cmp_in_sphere)

print 'JSD with Uniform :', jensen_shannon_divergence(random_grid_var, gt_grid_var)

/orions4-zfs/projects/optas/Git_Repos/pcloud_benchmark/evaluate_gan.py:55: UserWarning: Point-clouds not unit sphere.
  warnings.warn('Point-clouds not unit sphere.')


JSD with Uniform : 0.49069186393


In [12]:
# Compare JSD of GT with random noise.
mu, sigma = 0, 1
lower = -0.5
upper = 0.5
X = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
size = (len(gt_data), n_pc_samples, 3)
random_pcs = X.rvs(np.prod(size))
# random_pcs = random_pcs.reshape(size)
random_pcs = pclouds_centered_and_half_sphere(random_pcs)

_, random_grid_var = entropy_of_occupancy_grid(random_pcs, voxel_resolution, in_sphere=cmp_in_sphere)
print 'JSD with trunc-normal :', jensen_shannon_divergence(random_grid_var, gt_grid_var)

TypeError: object of type 'numpy.float64' has no len()